In [1]:
from pyspark import SparkContext,SparkConf
conf = SparkConf().setMaster("local").setAppName("Crimes")
sc = SparkContext(conf = conf)

In [2]:
# u.data 
lines = sc.textFile("/home/jovyan/work/Crimes/crimes.csv")
lines.take(10)

['ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location',
 '10508593,HZ250410,05/03/2016 09:00:00 PM,042XX W MADISON ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,SIDEWALK,true,false,1115,011,28,26,18,1148220,1899677,2016,05/10/2016 03:56:50 PM,41.880658176,-87.731212135,"(41.880658176, -87.731212135)"',
 '10508594,HZ250385,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,0810,THEFT,OVER $500,APARTMENT,false,true,0833,008,13,65,06,1149830,1860872,2016,05/10/2016 03:56:50 PM,41.77414081,-87.726307093,"(41.77414081, -87.726307093)"',
 '10508595,HZ250362,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,0560,ASSAULT,SIMPLE,STREET,false,true,0924,009,12,61,08A,1163662,1874660,2016,05/10/2016 03:56:50 PM,41.811697721,-87.675214316,"(41.811697721, -87.675214316)"',
 '10508597,HZ250202,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,0486,BATTERY,DOMESTIC BATTE

In [3]:
crimes_rdd = lines.map(lambda x: x.split(','))
crimes_rdd.take(10)

[['ID',
  'Case Number',
  'Date',
  'Block',
  'IUCR',
  'Primary Type',
  'Description',
  'Location Description',
  'Arrest',
  'Domestic',
  'Beat',
  'District',
  'Ward',
  'Community Area',
  'FBI Code',
  'X Coordinate',
  'Y Coordinate',
  'Year',
  'Updated On',
  'Latitude',
  'Longitude',
  'Location'],
 ['10508593',
  'HZ250410',
  '05/03/2016 09:00:00 PM',
  '042XX W MADISON ST',
  '1811',
  'NARCOTICS',
  'POSS: CANNABIS 30GMS OR LESS',
  'SIDEWALK',
  'true',
  'false',
  '1115',
  '011',
  '28',
  '26',
  '18',
  '1148220',
  '1899677',
  '2016',
  '05/10/2016 03:56:50 PM',
  '41.880658176',
  '-87.731212135',
  '"(41.880658176',
  ' -87.731212135)"'],
 ['10508594',
  'HZ250385',
  '05/03/2016 09:35:00 PM',
  '065XX S KEDVALE AVE',
  '0810',
  'THEFT',
  'OVER $500',
  'APARTMENT',
  'false',
  'true',
  '0833',
  '008',
  '13',
  '65',
  '06',
  '1149830',
  '1860872',
  '2016',
  '05/10/2016 03:56:50 PM',
  '41.77414081',
  '-87.726307093',
  '"(41.77414081',
  ' -87

In [4]:
columns = crimes_rdd.first()
columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [5]:
# tu veux que tes lignes soient différentes de ta première ligne
crimes_rdd = crimes_rdd.filter(lambda x : x != columns)

In [6]:
# calcul du nombre de crimes  par année
# crimesByYear = crimes_rdd.map(lambda x : (x[17],1)).reduceByKey(lambda x, y : x + y)

In [7]:
# for result in crimesByYear.collect():
#    print(result)

In [8]:
# nombre de crimes par type de crime (Primary Type)
crimesByType = crimes_rdd.map(lambda x : (x[5],1)).reduceByKey(lambda x, y : x + y)

In [9]:
for result in crimesByType.collect():
    print(result)

('PROSTITUTION', 1001)
('STALKING', 213)
('OBSCENITY', 66)
('SEX OFFENSE', 1101)
('OTHER NARCOTIC VIOLATION', 5)
('PUBLIC INDECENCY', 12)
('NARCOTICS', 15166)
('LIQUOR LAW VIOLATION', 283)
('ARSON', 624)
('CONCEALED CARRY LICENSE VIOLATION', 50)
('ROBBERY', 14541)
('CRIMINAL TRESPASS', 7898)
('HUMAN TRAFFICKING', 13)
('OTHER OFFENSE', 21286)
('MOTOR VEHICLE THEFT', 14101)
('WEAPONS VIOLATION', 4375)
('GAMBLING', 202)
('HOMICIDE', 909)
('THEFT', 74379)
('NON - CRIMINAL', 5)
('CRIM SEXUAL ASSAULT', 1775)
('CRIMINAL DAMAGE', 37412)
('OFFENSE INVOLVING CHILDREN', 2719)
('BURGLARY', 17335)
('ASSAULT', 22684)
('INTERFERENCE WITH PUBLIC OFFICER', 1172)
('KIDNAPPING', 237)
('DECEPTIVE PRACTICE', 21532)
('PUBLIC PEACE VIOLATION', 1935)
('NON-CRIMINAL', 52)
('INTIMIDATION', 163)
('BATTERY', 60706)
('NON-CRIMINAL (SUBJECT SPECIFIED)', 3)


In [10]:
crimesWithVehicle = crimes_rdd.filter(lambda x : ('VEHICLE' in x[5]) or ('VEHICLE' in x[6]))
crimesWithVehicle.count()

35772

In [11]:
nbCrimesTypeVehicle = crimes_rdd.filter(lambda row : "VEHICLE" in (row[5] or row[6]))
nbCrimesTypeVehicle.count()

14101

In [12]:
# le district avec le plus de crimes.       district: row[4]
    # 1) le nombre de crimes par district
    # 2) le max
    
crimesByDistrict = crimes_rdd.map(lambda row : (row[4],1)).reduceByKey(lambda x, y : x + y)

In [13]:
crimesByDistrict.sortBy(lambda x:x[1],False).collect()
# donc le district avec le plus de crimes est le district numéro 0486

[('0486', 30014),
 ('0820', 29061),
 ('0460', 19363),
 ('0810', 18293),
 ('1320', 18207),
 ('1310', 17240),
 ('0560', 14707),
 ('0890', 12165),
 ('0910', 11988),
 ('0860', 11110),
 ('0610', 10567),
 ('031A', 5806),
 ('1153', 5779),
 ('0620', 5597),
 ('2820', 5228),
 ('0320', 4455),
 ('1330', 4408),
 ('1811', 4333),
 ('1150', 4199),
 ('051A', 3573),
 ('2825', 3496),
 ('2826', 3306),
 ('143A', 3227),
 ('041A', 3096),
 ('2024', 2642),
 ('0870', 2395),
 ('0430', 2197),
 ('1130', 2071),
 ('4387', 2024),
 ('1154', 1999),
 ('0520', 1648),
 ('1152', 1632),
 ('1365', 1629),
 ('5002', 1591),
 ('0470', 1369),
 ('0497', 1314),
 ('0530', 1240),
 ('2014', 1225),
 ('2027', 1210),
 ('0281', 1150),
 ('1360', 1140),
 ('1305', 1134),
 ('4625', 1074),
 ('1121', 1071),
 ('1210', 1054),
 ('0420', 1031),
 ('502P', 970),
 ('2093', 958),
 ('1750', 931),
 ('1506', 913),
 ('0930', 909),
 ('0110', 905),
 ('0496', 879),
 ('0330', 877),
 ('0484', 877),
 ('0630', 848),
 ('1156', 831),
 ('0880', 797),
 ('0545', 744),

In [14]:
crimesByDistrict2 = crimes_rdd.map(lambda row : (row[4],1)).reduceByKey(lambda x, y : x + y)

In [15]:
crimesByDistrict2.max(lambda x: x[1])

('0486', 30014)